<a href="https://colab.research.google.com/github/AeroEng16/TennisPrediction/blob/main/dataPrepAndModelCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import math
import datetime
import seaborn as sns
import plotly.express as px
import ast
import plotly.graph_objects as go
from dateutil.relativedelta import relativedelta
#Imports to find best matchf for player without matching name
from difflib import SequenceMatcher
from operator import itemgetter

In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [10]:
url ="https://raw.githubusercontent.com/AeroEng16/TennisPrediction/main/subsetTrainingData_140724.csv"
unfiltered_df = pd.read_csv(url)
columnsToDrop = [
     'Player1_Aces',
 'Player1_BreakpointsWon',
 'Player1_DoubleFaults',
 'Player1_FirstServePointsWon',
 'Player1_FirstServeSuccessful',
 'Player1_MaxPointsInARow',
 'Player1_PointsWon',
 'Player1_SecondServePointsWon',
 'Player1_SecondServeSuccessful',
 'Player1_ServiceGamesWon',
 'Player1_ServicePointsLost',
 'Player1_ServicePointsWon',
 'Player1_TiebreaksWon',
 'Player1_TotalBreakpoints',
 'Player1_GamesWon',
 'Player1_MaxGamesInARow',
 'Player2_Aces',
 'Player2_BreakpointsWon',
 'Player2_DoubleFaults',
 'Player2_FirstServePointsWon',
 'Player2_FirstServeSuccessful',
 'Player2_MaxPointsInARow',
 'Player2_PointsWon',
 'Player2_SecondServePointsWon',
 'Player2_SecondServeSuccessful',
 'Player2_ServiceGamesWon',
 'Player2_ServicePointsLost',
 'Player2_ServicePointsWon',
 'Player2_TiebreaksWon',
 'Player2_TotalBreakpoints',
 'Player2_GamesWon',
 'Player2_MaxGamesInARow',
     'RatingCalculated',
     'matchID',
     'Player1',
     'Player2',
     'Unnamed: 0'
]
df = unfiltered_df.drop(columnsToDrop,axis=1)
df = unfiltered_df.drop([],axis=1)
df.head()

,Unnamed: 0,Date,Tournament,Court Type,Player1,Player1_Rank,Player1_RankingPoints,Player2,Player2_Rank,Player2_RankingPoints,...,Player2_ReturnPointsWonPercentage,Player2_SecondServePointsWonPercentage,Player2_SecondServeReturnPointsWonPercentage,Player2_ServiceGamesWonPercentage,Player2_ServicePointsWonPercentage,Player2_TotalPointsWonPercentage,Player2_WeightKg,percentageToVictory,percentageOfGamesWon,dominanceRatio
0,0,2024-05-16,Rome,red-clay,"Hurkacz, Hubert",9,3730,"Paul, Tommy",16,2300,...,38.0,53.0,49.0,75.0,62.0,50.0,82.0,"[0.083, 0.0]","[1.0, 0.0]","[1, 0.0]"
1,1,2024-05-16,Rome,red-clay,"Hurkacz, Hubert",9,3730,"Paul, Tommy",16,2300,...,38.0,53.0,49.0,75.0,62.0,50.0,82.0,"[0.083, 0.083]","[0.5, 0.5]","[1.0, 1.0]"
2,2,2024-05-16,Rome,red-clay,"Hurkacz, Hubert",9,3730,"Paul, Tommy",16,2300,...,38.0,53.0,49.0,75.0,62.0,50.0,82.0,"[0.083, 0.167]","[0.333, 0.667]","[0.5, 2.0]"
3,3,2024-05-16,Rome,red-clay,"Hurkacz, Hubert",9,3730,"Paul, Tommy",16,2300,...,38.0,53.0,49.0,75.0,62.0,50.0,82.0,"[0.167, 0.167]","[0.5, 0.5]","[1.0, 1.0]"
4,4,2024-05-16,Rome,red-clay,"Hurkacz, Hubert",9,3730,"Paul, Tommy",16,2300,...,38.0,53.0,49.0,75.0,62.0,50.0,82.0,"[0.167, 0.25]","[0.4, 0.6]","[0.667, 1.5]"


## Prep Dataset

### Split dataset into training, test and validation

In [29]:
# Percentage split for train test and validation

train_split, test_split, valid_split = 0.8,0.1,0.1

#Split by match ids not samples to give better representation

matchSamples = df.matchID.unique()

ADD SORTING FOR MATCH SAMPLES BY DATE

trainSplitIDs = matchSamples[0:int(len(matchSamples) * train_split)]
testSplitIDs = matchSamples[int(len(matchSamples) * train_split) :int(len(matchSamples) * (train_split+test_split))]
validSplitIDs = matchSamples[int(len(matchSamples) * (train_split+test_split)) :int(len(matchSamples) * (train_split+test_split+valid_split))]

len(trainSplitIDs),len(testSplitIDs),len(validSplitIDs)

train_df = df[df.matchID.isin(trainSplitIDs)]
test_df = df[df.matchID.isin(testSplitIDs)]
valid_df = df[df.matchID.isin(validSplitIDs)]

len(train_df), len(test_df), len(valid_df)

(6412, 741, 774)

In [35]:
min(valid_df.Date),max(test_df.Date),min(test_df.Date),max(train_df.Date)


('2024-04-15', '2024-04-19', '2024-04-16', '2024-05-16')

## Dataset Creation

In [ ]:
# Create the dataset

class tennisMatchDataset(Dataset):
  def __init__(self,csvDataFile,ignoredFeatures = None):
    self.df = pd.read_csv(csvDataFile)
    self.df = self.df.drop(ignoredFeatures)
  def __len__(self):
    return len(self.df)
  def __getitem__(self):
    self.label = df.Winner
    self.features = df.drop("Winner")
    return features, label